In [1]:
%pip install gtfs_kit
%pip install h3
%pip install keplergl

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [1]:
import sys
from pathlib import Path
import json
import h3

import pandas as pd
import numpy as np
import geopandas as gp
import matplotlib
import shapely as shp

import gtfs_kit as gk

## Read GTFS feeds
specify the paths to the zip files here

In [2]:
H3_RES = 11
# read GTFS data from multiple GTFS files
paths = ["../resources/rejseplanen.zip"]
feeds = []
for path in paths:
    feeds.append(gk.read_feed(path, dist_units='km'))

dates = ['20220320', '20220321', '20220322', '20220323', '20220324', '20220325', '20220326']

map all stops to their corresponding H3 cell

In [3]:
# map all stops to h3 for each feed
stop_h3_maps = []
for feed in feeds:
    feed.stops['h3'] = feed.stops.apply(lambda row: h3.geo_to_h3(row['stop_lat'],row['stop_lon'],H3_RES), axis=1)
    stop_h3_maps.append(pd.Series(feed.stops.h3.values,index=feed.stops.stop_id).to_dict())


### Export routes

In [3]:
%%capture --no-display
#save routes as geojson
#geojson = feed[0].routes_to_geojson(include_stops=True)
routes = gk.routes.geometrize_routes(feeds[0])
routes.head()
#with open("geojson.json", "w") as outfile:
#    json.dump(geojson, outfile)

/opt/conda/envs/myenv/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
/opt/conda/envs/myenv/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
/opt/conda/envs/myenv/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
/opt/conda/envs/myenv/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is depreca

,route_id,geometry,agency_id,route_short_name,route_long_name,route_type,route_color,route_text_color,route_desc
0,10001_3,"MULTILINESTRING ((12.28094 55.38463, 12.28139 ...",221,109,NaN,3,NaN,NaN,NaN
1,10033_3,"LINESTRING (11.32831 55.38977, 11.32859 55.389...",221,903,NaN,3,NaN,NaN,NaN
2,10163_3,"MULTILINESTRING ((12.22725 55.35119, 12.22729 ...",221,253,NaN,3,NaN,NaN,NaN
3,10164_3,"MULTILINESTRING ((12.22736 55.35099, 12.22743 ...",221,99N,NaN,3,NaN,NaN,NaN
4,10214_3,"LINESTRING (11.87714 54.76683, 11.87711 54.766...",221,741,NaN,3,NaN,NaN,NaN


In [10]:
def transform_geo(geo):
    if type(geo) == shp.geometry.LineString:
        return [[g[0], g[1]] for g in list(geo.coords)]
    else:
        return [[g[0], g[1]] for g in list(geo.geoms[0].coords)]

# routes_filtered = routes.loc[(routes['route_type'] < 5)]
routes_filtered = routes
routes_new = []

for idx, route in routes_filtered.iterrows():
    name = route["route_short_name"]
    category = route["route_type"]
    geo = route["geometry"]
    if type(geo) == shp.geometry.LineString:
        routes_new.append([name, category, [[g[0], g[1]] for g in list(geo.coords)]])
    else:
        for ge in geo.geoms:
            routes_new.append([name, category, [[g[0], g[1]] for g in list(ge.coords)]])

#routes_new['name'] = routes_filtered['route_short_name']
#routes_new['type'] = routes_filtered['route_type']
#routes_new['geo'] = routes_filtered['geometry'].apply(lambda geo: transform_geo(geo))

routes_df = pd.DataFrame(routes_new, columns= ['name', 'category', 'geo'])

routes_df.head()

,name,category,geo
0,109,3,"[[12.280937, 55.384628], [12.281387, 55.384157..."
1,109,3,"[[12.187489, 55.456978], [12.187485, 55.456924..."
2,109,3,"[[12.187489, 55.456978], [12.187485, 55.456924..."
3,903,3,"[[11.328311, 55.389772], [11.328595, 55.389757..."
4,253,3,"[[12.227247, 55.351191], [12.227285, 55.351262..."


In [47]:
routes_df.to_csv("routes.csv")

### Calculate Frequencies

In [4]:
# compute time series for each feed
ts = []
for feed in feeds:
    a = feed.compute_stop_time_series(dates=dates, freq='60Min').transpose().reset_index(level=1)
    ts.append(a)

In [5]:
def add_lists(lists):
    res = np.array([0]*len(lists[0]))
    for i in range(len(lists)-1):
        l = np.array(lists[i])
        l = np.nan_to_num(l)
        # print(f"adding res with {np.shape(res)} and list with {np.shape(l)}")
        res = np.add(res, l)
        # print(f"resulting shape: {np.shape(res)}")
    return res

In [6]:
result_df = pd.DataFrame()
for idx, feed in enumerate(feeds):
    tts = ts[idx]
    result_df['stop_id'] = tts['stop_id'].values.tolist()
    result_df['h3'] = result_df.apply(lambda row: stop_h3_maps[idx][row['stop_id']], axis=1)
    result_df['frequency'] = tts.iloc[:,1:].values.tolist()

# unique_h3s = set(stop_h3_maps[idx].values())
unique_h3s = [item for sublist in stop_h3_maps for item in sublist.values()]
h3_map = {}
for index in unique_h3s:
    res = result_df.loc[result_df['h3'] == index]
    if res.shape[0] > 1:
        res = add_lists(res['frequency'].values.tolist())
        h3_map[index] = res.tolist()
    elif res['frequency'].any():
        h3_map[index] = np.nan_to_num(res['frequency'].values[0]).tolist()
    

### Export Data

In [7]:
new = pd.DataFrame()
new['h3'] = h3_map.keys()
new['h3_group'] = new.apply(lambda row: h3.h3_to_parent(h = row['h3'], res=4), axis=1)
new['type'] = 0 # 0 => transit stop, as opposed to a regualr H3 cell, TODO: add distinction between types of transit stops
new['freq'] = h3_map.values()
new.head()

,h3,h3_group,type,freq
0,8b1f04d56803fff,841f04dffffffff,0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, ..."
1,8b1f23798753fff,841f237ffffffff,0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 3.0, ..."
2,8b1f23780099fff,841f237ffffffff,0,"[2.0, 1.0, 0.0, 2.0, 0.0, 2.0, 2.0, 4.0, 5.0, ..."
3,8b1f23683434fff,841f237ffffffff,0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 3.0, ..."
4,8b1f236b3132fff,841f237ffffffff,0,"[2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 5.0, ..."


In [8]:
h3_groups = list(set(new['h3_group']))

for h3_group in h3_groups:
    group = new.query(f"h3_group == '{h3_group}'")
    group[['h3', 'type', 'freq']].to_json(f"../docs/h3/{h3_group}.json", orient='records')

pd.DataFrame(h3_groups, columns= ["h3_group"]).to_json("../docs/h3.json", orient='values')